# When worlds collide even more: constrainted multi-objective optimization under uncertainty

In the previous optimization notebook ("freyberg_mou_1.ipynb"), we saw how we can use PESTPP-MOU to do some pretty killer decision support with multi-objective optimization.  It was awesome...

Now lets see how a stack, risk, and reliability can play here...


### Admin

Start off with the usual loading of dependencies and preparing model and PEST files. We will be continuing to work with the modified-Freyberg model (see "intro to model" notebook), and the high-dimensional PEST dataset prepared in the "pstfrom pest setup" and "obs and weights" notebooks. 

For the purposes of this notebook, you do not require familiarity with previous notebooks (but it helps...). 

Simply run the next few cells by pressing `shift+enter`.

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import numpy as np
import pandas as pd
font = {'family' : 'normal',
        'size'   : 15}
import matplotlib
matplotlib.rc('font', **font)
import matplotlib.pyplot as plt;
import shutil
import psutil

import sys
sys.path.insert(0,os.path.join("..", "..", "dependencies"))
import pyemu
import flopy
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
sys.path.insert(0,"..")
import herebedragons as hbd



To maintain continuity in the series of tutorials, we we use the PEST-dataset prepared in the "obs and weigths" tutorial. Run the next cell to copy fthe necessary files across. Note that if you will need to run the previous notebooks in the correct order beforehand.

Specify the path to the PEST dataset template folder. Recall that we will prepare our PEST dataset files in this folder, keeping them separate from the original model files. Then copy across pre-prepared model and PEST files:

In [ ]:
# specify the temporary working folder
t_d = os.path.join('freyberg6_template_chance')
if os.path.exists(t_d):
    shutil.rmtree(t_d)

org_t_d = os.path.join("freyberg6_template")
if not os.path.exists(org_t_d):
    raise Exception("you need to run the '/part2_8_opt/freyberg_opt_1.ipynb' notebook")

shutil.copytree(org_t_d,t_d)

In [ ]:
pst_path = os.path.join(t_d, 'freyberg_mf6.pst')

In [ ]:
pst = pyemu.Pst(pst_path)

## Stacks

So mechanically, how do we come up with this constraint PDF?  We saw previously in the PESTPP-IES notebook that we had to run the posterior parameter ensemble to yield a predictive PDF.  Well its no different here:  We will grab that PESTPP-IES posterior parameter ensemble (and manipulate it a little to remove decision variables) and then identify that ensemble as a "stack" of parameter realizations that can be run thru the model to yield constraint PDFs.  Easy as!  

Beware tho: including a stack in the optimization means we need to evaluate the stack at least once (see "coupling" below) which means we need queue up and run the stack along with the response matrix pertubation runs from before...lucky for you PESTPP-OPT does this automagically!

In [ ]:
# check that the pestpp-ies directory exists and that the posterior parameter ensemble exists
ies_dir = os.path.join("..","part2_6_ies","master_ies_1")
if not os.path.exists(ies_dir):
    raise Exception("you need to run the 'part2_6_ies/freyberg_ies_1_basics.ipynb' notebook")

In [ ]:
pe_files = [f for f in os.listdir(ies_dir) if f.endswith(".par.csv") and f.startswith("freyberg_mf6")]
pe_files.sort()
pe_files
pe = pd.read_csv(os.path.join(ies_dir,pe_files[-1]),index_col=0)

Now load the parameter ensemble from the last iteration of PESTPP-IES:

In [ ]:
pe = pd.read_csv(os.path.join(ies_dir,pe_files[-1]),index_col=0)
pe

In [ ]:
par = pst.parameter_data
par.loc[par.partrans=="fixed","partrans"] = "none"
wpar = par.loc[par.parnme.str.contains("wel") & par.parnme.str.contains("cn"),"parnme"]
pe.loc[:,wpar.values] = 1.0

In [ ]:
pe.to_csv(os.path.join(t_d,"par_stack.csv"))

In [ ]:
risk_tpl = os.path.join(t_d,"risk.dat.tpl")
with open(risk_tpl,'w') as f:
          f.write("ptf ~\n")
          f.write("risk ~ _risk_~ \n")
pst.add_parameters(risk_tpl,pst_path=".")
par = pst.parameter_data
par.loc["_risk_","partrans"] = "none"
par.loc["_risk_","parubnd"] = 0.999
par.loc["_risk_","parlbnd"] = 0.001
par.loc["_risk_","parval1"] = 0.5
par.loc["_risk_","pargp"] = "decvars"


pst.add_pi_equation(["_risk_"],pilbl="risk_eq",obs_group="greater_than")


In [ ]:
pst.prior_information

In [ ]:
dv_file = os.path.join(t_d,"initial_dvpop.csv")
mou1_m_d = os.path.join("master_mou_1")
if os.path.exists(mou1_m_d):
    files = [f for f in os.listdir(mou1_m_d) if f.endswith("dv_pop.csv") and "archive" not in f]
    files.sort()
    dv_file = os.path.join(mou1_m_d,files[-2])
print(dv_file)
df = pd.read_csv(dv_file,index_col=0)
df.loc[:,"_risk_"] = np.random.uniform(0.001,0.999,df.shape[0])
df.to_csv(dv_file)

In [ ]:
pst.pestpp_options["mou_objectives"] += ",_risk_"
pst.pestpp_options["mou_risk_objective"] = True
pst.pestpp_options["opt_recalc_chance_every"] = 100000

In [ ]:
pst.pestpp_options["opt_par_stack"] = "par_stack.csv"
pst.pestpp_options["opt_risk"] = 0.95

In [ ]:
obs_org = pst.observation_data.copy()
obs = pst.observation_data
#obs.loc[obs.apply(lambda x: x.weight > 0 and "wel" in x.obsnme,axis=1),"weight"] = 0.0


In [ ]:
pst.noptmax = 100
pst.write(pst_path,version=2)

## An aside on "coupling": interaction between decision variables, parameters, and constraints

blahblahblah




# Attention!

You must specify the number which is adequate for ***your*** machine! Make sure to assign an appropriate value for the following `num_workers` variable:

In [ ]:
num_workers = 22 #psutil.cpu_count(logical=False) # update according to your available resources!

Then specify the folder in which the PEST manager will run and record outcomes. It should be different from the `t_d` folder. 

In [ ]:
m_d = os.path.join('master_opt_2')

The following cell deploys the PEST agents and manager and then starts the run using `pestpp-opt`. Run it by pressing `shift+enter`.

If you wish to see the outputs in real-time, switch over to the terminal window (the one which you used to launch the `jupyter notebook`). There you should see `pestpp-opt`'s progress. 

If you open the tutorial folder, you should also see a bunch of new folders there named `worker_0`, `worker_1`, etc. These are the agent folders. `pyemu` will remove them when PEST finishes running.

This run should take a while to complete (depending on the number of workers and the speed of your machine). If you get an error, make sure that your firewall or antivirus software is not blocking `pestpp-opt` from communicating with the agents (this is a common problem!).

In [ ]:
pyemu.os_utils.start_workers(t_d,"pestpp-mou","freyberg_mf6.pst",num_workers=num_workers,worker_root=".",
                           master_dir=m_d)